# Loss Functions for ASR
Automatic Speech Recognition (ASR) systems are designed to convert spoken language into text. These systems rely heavily on deep learning models that require specific types of loss functions to effectively learn from the audio data. Loss functions measure the discrepancy between the model's predictions and the actual target outputs. In ASR, we face the challenge of aligning variable-length speech input sequences to their corresponding text transcripts. Traditional loss functions aren't directly applicable in this scenario, resulting in specialized loss functions like CTC and cross-entropy are crucial.

### **1. Connectionist Temporal Classification (CTC) Loss**

CTC Loss is designed for sequence prediction tasks where the timing of the output is variable and not aligned with the input sequence (e.g., speech audio frames to text mapping). CTC operates by summing the probabilities of all possible alignments between the input sequences and the target sequences that could result in a correct prediction, making it highly effective for tasks like ASR where the alignment between the audio and transcript is not known in advance.

An excellent and more detailed explanation of CTC loss can be [here](https://distill.pub/2017/ctc/)

#### Why CTC Loss is need for ASR?

**Aligning Mismatched Lengths**: Speech segments corresponding to single characters in the transcript can vary in length. CTC addresses this by introducing a "blank" token, allowing the model to output repeated characters or blanks, making the output sequence as long as the input.

**No Pre-Segmentation**: CTC works without needing the exact location of each character within the audio, thus avoiding complex pre-segmentation of the input.

<img src="./imgs/ctc.webp" alt="drawing" width="650"/>

CTC allows for the training of end-to-end systems without requiring pre-segmented training data, which simplifies the model training process and improves the ability to handle varying input lengths and misalignments between the spoken audio and text.

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim

# Create a simple model with LSTM for handling sequence data
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(SimpleNN, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # LSTM output shape is (batch, sequence, feature)
        outputs, _ = self.lstm(x)
        # Reshape output for the fully connected layer
        outputs = self.fc(outputs)
        # Apply log softmax on the last dimension (num_classes)
        return outputs.log_softmax(2)

# Initialize the model, loss function, and optimizer
input_size = 13  # Typically the number of MFCCs used as features
hidden_size = 128
num_layers = 2
num_classes = 20  # Including the blank label for CTC
model = SimpleNN(input_size, hidden_size, num_layers, num_classes)
ctc_loss = nn.CTCLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Generate some random data
batch_size = 16
sequence_length = 50
inputs = torch.randn(batch_size, sequence_length, input_size)  # (batch, sequence, feature)
input_lengths = torch.full((batch_size,), sequence_length, dtype=torch.long)
targets = torch.randint(1, num_classes, (batch_size, 30), dtype=torch.long)
target_lengths = torch.randint(10, 30, (batch_size,), dtype=torch.long)

# Forward pass: compute predicted outputs by passing inputs to the model
logits = model(inputs)  # (batch, sequence, num_classes)
logits = logits.transpose(0, 1)  # CTC needs input as (sequence, batch, num_classes)

# Calculate loss
loss = ctc_loss(logits, targets, input_lengths, target_lengths)

# Backward pass: compute gradient of the loss with respect to model parameters
loss.backward()

# Perform a single optimization step (parameter update)
optimizer.step()

print("CTC Loss:", loss.item())


CTC Loss: 5.795548439025879


The `CTCLoss` function is used to compute the loss between the predictions (inputs) and the true labels (targets). This function takes into account the lengths of each sequence, allowing it to handle the common scenario in ASR where input and output lengths do not match directly.

### **2. Cross-Entropy (CE) Loss**

Cross-Entropy Loss is used for classification tasks. It measures the performance of a classification model whose output is a probability value between 0 and 1. Cross-entropy loss increases as the predicted probability diverges from the actual label, making it ideal for models where predictions need to be probabilities that sum to one.

#### Why Cross-Entropy Loss is need for ASR?

**Frame-level Classification:** CE loss is suited for models that predict the probability distribution over possible output characters at each timestep of the input and helps in maximizing the likelihood of the correct labels in multi-class settings 

**Use with Attention/Hybrid Models:** CE loss is often combined with attention-based or hybrid ASR models like Transformer-based systems.

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim

# Create a simple model
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleNN, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        return self.linear(x)

# Initialize the model, loss function, and optimizer
input_size = 10
num_classes = 5
model = SimpleNN(input_size, num_classes)
cross_entropy_loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Generate some random data
inputs = torch.randn(10, input_size)  # batch size is 10
targets = torch.randint(0, num_classes, (10,))

# Forward pass: compute predicted outputs by passing inputs to the model
logits = model(inputs)

# Calculate loss
loss = cross_entropy_loss(logits, targets)

# Backward pass: compute gradient of the loss with respect to model parameters
loss.backward()

# Perform a single optimization step (parameter update)
optimizer.step()

print("Cross-Entropy Loss:", loss.item())


Cross-Entropy Loss: 1.7233741283416748


The `CrossEntropyLoss` function internally applies a softmax to the logits before calculating the loss, comparing the predicted probabilities directly against the true labels in a one-hot encoded fashion.

### CTC vs Cross-Entropy Loss

| **Aspect**              | **CTC Loss**                                                                                                                                                           | **Cross-Entropy Loss**                                                                                                                                                        |
|-------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------:|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|
| **Problem Framing**     | ASR as a sequence-to-sequence mapping without explicit alignment.                                                                                                      | ASR as a classification problem for each time step, or as a next-step prediction problem.                                                                                    |
| **Nature of Task**      | Designed for tasks where the alignment between input sequences and output sequences is not known and is variable.                                                      | Used when each output from the model is a classification problem over a fixed set of classes.                                                                                |
| **Use Case**            | Ideal for models where you do not have or do not want to rely on pre-segmented data. The model learns to map input sequences to output labels directly.                  | Often used in models that predict the next character or word based on the previous context or classify each audio frame into a phoneme or word.                              |
| **Advantage**           | Efficiently handles varying lengths of inputs and outputs. Can output a condensed sequence of predicted labels, removing duplicates and blanks.                         | Straightforward to implement and interpret, especially when there is a clear and fixed correspondence between input features and labels at each time step.                    |
| **Typical Use**         | Used in end-to-end ASR models where raw audio input is converted directly into text without needing manual segmentation into words or phonemes.                         | Suitable for systems where audio frames are aligned with specific classes using another method and the model is trained to classify each frame or predict the next sequence. |
| **Examples of State-of-the-Art (SOTA) Models** | DeepSpeech, Wav2Vec | Whisper  |
